# Creating Config Files

This notebook generates config files that specify supernova specific priors 
and fitting arguments. Individual files are created for different supernova 
surveys. The config files are save in yaml format and values typically follow the 
following template:

    hsiao_x1:
      object_id:
        kwargs:
          bounds:
            t0:
              - ? # Typically t0 - 10
              - ? # Typically t0 + 10
            x1:
              - -1
              - 1
          phase_range:
            - -20
            - 50
        priors:
          z: ?
          t0: ?

    sn91bg:
      object_id:
        kwargs:
          bounds:
            c:
              - 0
              - 1
            x1:
              - 0.65
              - 1.25
            t0:
              - ? # Typically t0 - 10
              - ? # Typically t0 + 10
          phase_range:
            - -20
            - 50
        priors:
          z: ?
          t0: ?

We note that values for some supernovae may have been manually altered in the
config files after manually inspecting their  light curve fits. For this reason,
an existing config file should not be overwritten on the the assumption that
the automatically generated and currently existing config files are the same.

#### Table of Contents:
1. <a href='#sdss'>SDSS - Sako et al. 2018</a>: Generate a config file for the Sloan Digital Sky Survey
1. <a href='#csp'>CSP - DR1</a>: Generate a config file for the Carnegie Supernova Project
1. <a href='#des'>DES - SN3yr</a>: Generate a config file for the Dark Energy Survey


In [ ]:
from pathlib import Path

import numpy as np
import yaml
from sndata.csp import dr1
from sndata.des import sn3yr
from sndata.sdss import sako18

out_dir = Path('../config_files')
out_dir.mkdir(exist_ok=True, parents=True)


In [ ]:
def create_config_dict(id_arr, t0_arr, z_arr, t0_range=10):
    """Create a dictionary with fitting params and kwargs for our pipeline
    
    Args:
        id_add (ndarray): An array of object Ids
        t0_arr (ndarray): An array of t0 values for each object
        z_arr  (ndarray): An array of redshift values for each object
        t0_range (float): The + / - bounds when fitting t0
    """
    
    config_dict = {'hsiao_x1': {}, 'sn91bg': {}}
    for obj_id, t0, z in zip(id_arr, t0_arr, z_arr):

        config_dict['hsiao_x1'][obj_id] = {'priors': {}, 'kwargs': {}}
        config_dict['hsiao_x1'][obj_id]['priors'] = {'z': z, 't0': t0}
        config_dict['hsiao_x1'][obj_id]['kwargs'] = {'bounds': {}, 'phase_range': {}}
        config_dict['hsiao_x1'][obj_id]['kwargs']['bounds'] = {'t0': [t0 - 10, t0 + 10], 'x1': [-1, 1]}
        config_dict['hsiao_x1'][obj_id]['kwargs']['phase_range'] = [-20, 50]

        config_dict['sn91bg'][obj_id] = {'priors': {}, 'kwargs': {}}
        config_dict['sn91bg'][obj_id]['priors'] = {'z': z, 't0': t0}
        config_dict['sn91bg'][obj_id]['kwargs'] = {'bounds': {}, 'phase_range': {}}
        config_dict['sn91bg'][obj_id]['kwargs']['bounds'] = {'t0': [t0 - 10, t0 + 10], 'c': [0, 1], 'x1': [0.65, 1.25]}
        config_dict['sn91bg'][obj_id]['kwargs']['phase_range'] = [-20, 50]
    
    return config_dict

def raise_path_exists(path):
    """Raise an error if a path exists
    
    Args:
        path (Path): The path to check
    """

    if path.exists():
        raise ValueError(
            'Existing files may have manually modified values.'
            ' Be careful not to overwrite them!'
        )
    

## SDSS - Sako et al. 2018 <a id='SDSS'></a>

In [ ]:
master = sako18.load_table('master').to_pandas()
master['JD'] = master['MJDatPeakrmag'] + 2400000.5
master.head()


In [ ]:
targets_with_redshift = master[master['zCMB'] >= 0]

print('Total targets of with z:', len(targets_with_redshift), '/', len(master))


In [ ]:
sdss_config = create_config_dict(
    targets_with_redshift['CID'],
    targets_with_redshift['JD'], 
    targets_with_redshift['zCMB'])


In [ ]:
sdss_out_path = out_dir / Path('sdss_config.yml')
raise_path_exists(sdss_out_path)
with open(sdss_out_path, 'w') as ofile:
    yaml.dump(sdss_config, ofile)


## CSP - DR1 <a id='CSP'></a>

In [ ]:
csp_table_1 = dr1.load_table(1).to_pandas()
csp_table_1['JD'] = csp_table_1['T0'] + 2400000.5
csp_table_1.head()


In [ ]:
csp_config = create_config_dict(
    csp_table_1['SN'],
    csp_table_1['JD'], 
    csp_table_1['zCMB'])



In [ ]:
csp_out_path = out_dir / Path('csp_config.yml')
raise_path_exists(csp_out_path)
with open(csp_out_path, 'w') as ofile:
    yaml.dump(csp_config, ofile)


## DES - SN3yr <a id='DES'></a>

In [ ]:
des_fit_res = sn3yr.load_table('SALT2mu_DES+LOWZ_C11.FITRES').to_pandas()
des_fit_res['JD'] = des_fit_res['PKMJD'] + 2400000.5
des_fit_res.CIDint = des_fit_res.CIDint.apply(lambda x: str(x).zfill(8))
des_fit_res.head()


In [ ]:
des_config = create_config_dict(
    des_fit_res['CIDint'],
    des_fit_res['JD'], 
    des_fit_res['zCMB'])


In [ ]:
des_out_path = out_dir / Path('des_config.yml')
raise_path_exists(des_out_path)
with open(des_out_path, 'w') as ofile:
    yaml.dump(des_config, ofile)
    